### Libraries

In [1]:
import os
import glob
import polars as pl
import pandas as pd
from pandas.io.json._normalize import json_normalize
import json 

## Table 1: Sales/NFT

* This serves as a engineered features from sales table that contains information of last buyer, last second buyer, and last seller. 
* Table 1 also includes prices of each NFT given different time, first sale, last sale, and last second sale. 
* Table 1 also has time of each event mentioned above

In [ ]:


# Set up JSON directory and file pattern.
json_dir = r"/Volumes/SAKIV/Research/NFT/Project/NFT_Event_Sale"
file_pattern = os.path.join(json_dir, 'NFT_*_*.json')

all_events = []        # List to store flattened event dictionaries.
tokens_without_events = []  # List to store tokens with no events.

for filepath in glob.glob(file_pattern):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
            # If no events exist, store the token id
            if not data.get('events'):
                tokens_without_events.append(data['metadata']['token_id'])
            else:
                # For each event, create a flattened record by merging event info with metadata.
                for event in data['events']:
                    # You can use dict unpacking to merge dictionaries.
                    flattened_record = {
                        # Event-level data (assumes event is a dict; update keys as needed)
                        **event,
                        # Metadata fields
                        'nft_identifier': data['metadata']['token_id'],
                        'nft_name': data['metadata']['nft_name'],
                        'event_count': data['metadata']['event_count'],
                        'timestamp': data['metadata']['timestamp']
                    }
                    all_events.append(flattened_record)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from file {filepath}: {e}")
    except Exception as e:
        print(f"An error occurred with file {filepath}: {e}")

# Create a Polars DataFrame directly from the list of flattened dictionaries.
df_events_pl = pl.DataFrame(all_events)

print(f"Number of NFTs with no events: {len(tokens_without_events)}")
print("Tokens with no events:", tokens_without_events)
print("Number of events loaded:", df_events_pl.height)
print(df_events_pl.head())


Number of NFTs with no events: 744
Tokens with no events: ['0', '1010', '1014', '1016', '1019', '1023', '1028', '1031', '1034', '1035', '1037', '104', '1077', '1079', '107', '1081', '1089', '1093', '1101', '110', '1110', '1114', '1117', '111', '1120', '1122', '1123', '1135', '113', '1149', '116', '1195', '1196', '121', '1229', '1258', '1259', '1262', '1263', '1276', '1288', '12', '1313', '1316', '1324', '1328', '1341', '1342', '1356', '1358', '1359', '1362', '1372', '1382', '1386', '1387', '1394', '1417', '1419', '1431', '143', '1446', '1452', '1468', '1474', '1475', '148', '14', '1506', '1532', '1538', '153', '155', '1586', '1596', '1597', '1619', '1625', '1639', '164', '1652', '1655', '165', '1664', '1665', '1666', '1667', '1668', '1669', '1670', '1671', '1672', '1673', '1674', '1675', '1677', '1683', '1684', '1745', '1777', '1820', '187', '1914', '1966', '1967', '1968', '1982', '1991', '1992', '1995', '1', '2034', '2035', '2038', '2039', '2054', '2090', '2107', '2114', '2147', '2156

In [3]:
print(df_events_pl.columns)

['event_type', 'order_hash', 'chain', 'protocol_address', 'closing_date', 'nft', 'quantity', 'seller', 'buyer', 'payment', 'transaction', 'event_timestamp', 'nft_identifier', 'nft_name', 'event_count', 'timestamp']


In [4]:
# Create the DataFrame using our list of flattened records (all_events)
df_events_pl = pl.DataFrame(all_events)  # or pl.from_records(all_events)

print("Type of df_events_pl:", type(df_events_pl))
print("Columns:", df_events_pl.columns)


Type of df_events_pl: <class 'polars.dataframe.frame.DataFrame'>
Columns: ['event_type', 'order_hash', 'chain', 'protocol_address', 'closing_date', 'nft', 'quantity', 'seller', 'buyer', 'payment', 'transaction', 'event_timestamp', 'nft_identifier', 'nft_name', 'event_count', 'timestamp']


In [5]:
# A new DataFrame with the filtered information 

# create a new feature that is the first event timestamp for each NFT
# crate a new feature that is the second event timestamp for each NFT
# create a new feature that is the last event timestamp for each NFT
# create a new feature that is the price(quantity) divided by 10^18 of first event for each NFT
# create a new feature that is the price(quantity) divided by 10^18 of second event for each NFT
# create a new feature that is the price(quantity) divided by 10^18 of last event for each NFT
# create a new feature that is the buyer of first event for each NFT
# create a new feature that is the seller of first event for each NFT 
# create a new feature that is the buyer of second event for each NFT

df_events_pl = df_events_pl.with_columns(pl.col("nft_identifier").alias("token_id"))

# 
df_table1 = df_events_pl.group_by("token_id").agg([
    pl.col("event_timestamp").first().alias("time_n_sale"),
    pl.col("event_timestamp").shift(-1).get(0).alias("time_n-1_sale"),
    pl.col("event_timestamp").shift(-2).get(0).alias("time_n-2_sale"),
    pl.col("event_timestamp").last().alias("time_1_sale"),
    (pl.col("payment").struct.field("quantity").cast(pl.Float64) / 10**18).first().alias("price_n_sale"),
    (pl.col("payment").struct.field("quantity").cast(pl.Float64) / 10**18).shift(-1).get(0).alias("price_n-1_sale"),
    (pl.col("payment").struct.field("quantity").cast(pl.Float64) / 10**18).shift(-2).get(0).alias("price_n-2_sale"),
    (pl.col("payment").struct.field("quantity").cast(pl.Float64) / 10**18).last().alias("price_1_sale"),
    pl.col("buyer").first().alias("buyer_n_sale"),
    pl.col("seller").first().alias("seller_n_sale"),
    pl.col("buyer").shift(-1).get(0).alias("buyer_n-1_sale"),
    pl.col("seller").shift(-1).get(0).alias("seller_n-1_sale"),
    pl.col("event_count").first().alias("event_count")
])


print(df_table1.head(10))


shape: (10, 14)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ token_id ┆ time_n_sa ┆ time_n-1_ ┆ time_n-2_ ┆ … ┆ seller_n_ ┆ buyer_n-1 ┆ seller_n- ┆ event_cou │
│ ---      ┆ le        ┆ sale      ┆ sale      ┆   ┆ sale      ┆ _sale     ┆ 1_sale    ┆ nt        │
│ str      ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│          ┆ i64       ┆ i64       ┆ i64       ┆   ┆ str       ┆ str       ┆ str       ┆ i64       │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 9252     ┆ 170977921 ┆ 162955281 ┆ null      ┆ … ┆ 0xf15c935 ┆ 0xe876387 ┆ 0x228c133 ┆ 2         │
│          ┆ 1         ┆ 1         ┆           ┆   ┆ 62bc3944a ┆ b8246663b ┆ 4fc57eb6e ┆           │
│          ┆           ┆           ┆           ┆   ┆ 68e938ef7 ┆ c0f3c7611 ┆ 02ecc448e ┆           │
│          ┆           ┆           ┆           ┆   ┆ 5d2…      ┆ 4ad…      

In [ ]:
# Save df_table1 to a CSV file
# path to save the CSV file
analysis_dir = r"/Volumes/SAKIV/Research/NFT/DataAnalysis/Model/Analysis_NFT_Sales/Final"
# analysis_dir = r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\\"

csv_path = os.path.join(analysis_dir, 'df_table1.csv')
df_table1.write_csv(csv_path, include_header=True, separator=',')
print(f"DataFrame saved to {csv_path}")

DataFrame saved to /Volumes/SAKIV/Research/NFT/DataAnalysis/Model/Analysis_NFT_Sales/Final/df_table1.csv


In [36]:
import polars as pl

# Filter the DataFrame for nft_identifier == "23"
df_token_23 = df_events_pl.filter(pl.col("nft_identifier") == "23")

# Display the resulting DataFrame
print(df_token_23)




shape: (3, 17)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ eve ┆ ord ┆ cha ┆ pro ┆ clo ┆ nft ┆ qua ┆ sel ┆ buy ┆ pay ┆ tra ┆ eve ┆ nft ┆ nft ┆ eve ┆ tim ┆ tok │
│ nt_ ┆ er_ ┆ in  ┆ toc ┆ sin ┆ --- ┆ nti ┆ ler ┆ er  ┆ men ┆ nsa ┆ nt_ ┆ _id ┆ _na ┆ nt_ ┆ est ┆ en_ │
│ typ ┆ has ┆ --- ┆ ol_ ┆ g_d ┆ str ┆ ty  ┆ --- ┆ --- ┆ t   ┆ cti ┆ tim ┆ ent ┆ me  ┆ cou ┆ amp ┆ id  │
│ e   ┆ h   ┆ str ┆ add ┆ ate ┆ uct ┆ --- ┆ str ┆ str ┆ --- ┆ on  ┆ est ┆ ifi ┆ --- ┆ nt  ┆ --- ┆ --- │
│ --- ┆ --- ┆     ┆ res ┆ --- ┆ [14 ┆ i64 ┆     ┆     ┆ str ┆ --- ┆ amp ┆ er  ┆ str ┆ --- ┆ str ┆ str │
│ str ┆ str ┆     ┆ s   ┆ i64 ┆ ]   ┆     ┆     ┆     ┆ uct ┆ str ┆ --- ┆ --- ┆     ┆ i64 ┆     ┆     │
│     ┆     ┆     ┆ --- ┆     ┆     ┆     ┆     ┆     ┆ [4] ┆     ┆ i64 ┆ str ┆     ┆     ┆     ┆     │
│     ┆     ┆     ┆ str ┆     ┆     ┆     ┆     ┆     ┆     ┆     ┆     ┆     ┆     ┆     ┆     ┆     │
╞═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪

In [37]:

df_token_23_stats = df_token_23.select([
    pl.col("event_timestamp").first().alias("time_n_sale"),
    pl.col("event_timestamp").shift(-1).get(0).alias("time_n-1_sale"),
    pl.col("event_timestamp").last().alias("time_1_sale"),
    (pl.col("payment").struct.field("quantity").cast(pl.Float64) / 10**18).first().alias("priece_n_sale"),
    (pl.col("payment").struct.field("quantity").cast(pl.Float64) / 10**18).shift(-1).get(0).alias("price_n-1_sale"),
    (pl.col("payment").struct.field("quantity").cast(pl.Float64) / 10**18).last().alias("price_1_sale"),
    pl.col("buyer").first().alias("buyer_n_sale"),
    pl.col("seller").first().alias("seller_n_sale"),
    pl.col("buyer").shift(-1).get(0).alias("buyer_n-1_sale"),
    pl.col("seller").shift(-1).get(0).alias("seller_n-1_sale"),
    pl.col("event_count").first().alias("event_count")
])

# do not truncate the output
pl.Config.set_tbl_cols(1000)

# do not truncate the output in each cell 
pl.Config.set_tbl_rows(1000)


# Display the statistics
print(df_token_23_stats)

shape: (1, 11)
┌────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┐
│ time_n ┆ time_n ┆ time_1 ┆ priece ┆ price_ ┆ price_ ┆ buyer_ ┆ seller ┆ buyer_ ┆ seller ┆ event_ │
│ _sale  ┆ -1_sal ┆ _sale  ┆ _n_sal ┆ n-1_sa ┆ 1_sale ┆ n_sale ┆ _n_sal ┆ n-1_sa ┆ _n-1_s ┆ count  │
│ ---    ┆ e      ┆ ---    ┆ e      ┆ le     ┆ ---    ┆ ---    ┆ e      ┆ le     ┆ ale    ┆ ---    │
│ i64    ┆ ---    ┆ i64    ┆ ---    ┆ ---    ┆ f64    ┆ str    ┆ ---    ┆ ---    ┆ ---    ┆ i64    │
│        ┆ i64    ┆        ┆ f64    ┆ f64    ┆        ┆        ┆ str    ┆ str    ┆ str    ┆        │
╞════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╡
│ 164168 ┆ 162272 ┆ 162213 ┆ 289.0  ┆ 23.0   ┆ 2.0    ┆ 0x73d8 ┆ 0x7f8b ┆ 0xf7ee ┆ 0xa0fe ┆ 3      │
│ 7735   ┆ 7617   ┆ 3770   ┆        ┆        ┆        ┆ 55f11c ┆ 045794 ┆ 6c2f81 ┆ c1a6b8 ┆        │
│        ┆        ┆        ┆        ┆        ┆        ┆ bc0379 ┆ 1ea2df ┆ 1b

## Table 2: Transfer Table 

* Table 2 includes the transfer events happened between the second last sale to the last sale. 

* Imagine one sells the NFT, they must own the NFT first. The time window between the previous purchase and last sale would be a good window to drive price up. Therefore, table 2 is constructed to only capture the transfer actions of each NFT after they were sold for the second last time. 
* 

In [ ]:

# Set up your JSON directory and file pattern.
json_dir = r"C:\Emory\Research\NFT\Project\NFT_Event_Transfer\\"
file_pattern = os.path.join(json_dir, 'NFT_*_*.json')

all_transfer_events = []        # List to store flattened event dictionaries.
tokens_without_transfer_events = []  # List to store tokens with no events.

for filepath in glob.glob(file_pattern):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
            # If no events exist, store the token id
            if not data.get('events'):
                tokens_without_events.append(data['metadata']['token_id'])
            else:
                # For each event, create a flattened record by merging event info with metadata.
                for event in data['events']:
                    # You can use dict unpacking to merge dictionaries.
                    flattened_transfer_record = {
                        # Event-level data (assumes event is a dict; update keys as needed)
                        **event,
                        # Metadata fields
                        'nft_identifier': data['metadata']['token_id'],
                        'nft_name': data['metadata']['nft_name'],
                        'event_count': data['metadata']['event_count'],
                        'timestamp': data['metadata']['timestamp']
                    }
                    all_transfer_events.append(flattened_transfer_record)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from file {filepath}: {e}")
    except Exception as e:
        print(f"An error occurred with file {filepath}: {e}")

# Create a Polars DataFrame directly from the list of flattened dictionaries.
df_transfer_events_pl = pl.DataFrame(all_transfer_events)

print(f"Number of NFTs with no events: {len(tokens_without_transfer_events)}")
print("Tokens with no events:", tokens_without_transfer_events)
print("Number of events loaded:", df_transfer_events_pl.height)
print(df_transfer_events_pl.head())


Number of NFTs with no events: 0
Tokens with no events: []
Number of events loaded: 220465
shape: (5, 12)
┌────────┬────────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ event_ ┆ chain  ┆ transa ┆ from_ ┆ to_ad ┆ quant ┆ nft   ┆ event ┆ nft_i ┆ nft_n ┆ event ┆ times │
│ type   ┆ ---    ┆ ction  ┆ addre ┆ dress ┆ ity   ┆ ---   ┆ _time ┆ denti ┆ ame   ┆ _coun ┆ tamp  │
│ ---    ┆ str    ┆ ---    ┆ ss    ┆ ---   ┆ ---   ┆ struc ┆ stamp ┆ fier  ┆ ---   ┆ t     ┆ ---   │
│ str    ┆        ┆ str    ┆ ---   ┆ str   ┆ i64   ┆ t[14] ┆ ---   ┆ ---   ┆ str   ┆ ---   ┆ str   │
│        ┆        ┆        ┆ str   ┆       ┆       ┆       ┆ i64   ┆ str   ┆       ┆ i64   ┆       │
╞════════╪════════╪════════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╡
│ transf ┆ ethere ┆ 0x8a65 ┆ 0xaba ┆ 0xf78 ┆ 1     ┆ {"0", ┆ 16198 ┆ 0     ┆ 0     ┆ 2     ┆ 2025- │
│ er     ┆ um     ┆ c8db39 ┆ 7161a ┆ 01b81 ┆       ┆ "bore ┆ 17106 ┆       ┆       ┆  

In [26]:
print(df_transfer_events_pl.columns)

['event_type', 'chain', 'transaction', 'from_address', 'to_address', 'quantity', 'nft', 'event_timestamp', 'nft_identifier', 'nft_name', 'event_count', 'timestamp']


In [27]:
# for given users that are foundable in df_table1 column: seller_n_sale and buyer_n-1_sale
# start looking for the token_id that has been transfered starting from buyer_n-1_sale and each row start with that token_id
# so the first entry of start of every nft is by that buyer 
# this will continue going for example buyer_n-1_sale is x1 and it transfer to x2, this will be the first record 
# then x2 transfer to x3, this is the second record
# x3 transfer to x4, this is the third record, and if x4 is the seller_n_sale then we stop looking for the token_id
# this dataframe will have four columns: token_id, transfer_from, transfer_to, event_timestamp 

# Create a new version of the DataFrame with the required columns added as aliases
df_transfer_events_pl = df_transfer_events_pl.with_columns([
	pl.col("nft_identifier").alias("token_id"),
	pl.col("from_address").alias("transfer_from"),
	pl.col("to_address").alias("transfer_to")
])


In [40]:
import polars as pl

# Helper function to build the transfer chain for a single token.
def build_transfer_chain(events: pl.DataFrame, token_id: str, start_pair: tuple, stop_pair: tuple) -> pl.DataFrame:
    """
    Given a Polars DataFrame of transfer events for one token (sorted by event_timestamp),
    build a chain of transfers.
    
    Parameters:
      events: A Polars DataFrame containing events for one token.
      token_id: The token identifier.
      start_pair: A tuple (start_from, start_to) where start_from is the expected 
                  "from_address" (seller_n-1_sale) and start_to is the expected "to_address" (buyer_n-1_sale).
      stop_pair: A tuple (stop_from, stop_to) that signals the end of the chain (seller_n_sale, buyer_n_sale).
      
    Returns:
      A Polars DataFrame with columns: token_id, transfer_from, transfer_to, event_timestamp.
      The chain starts from the event that matches the start_pair and continues until the stop_pair is encountered.
    """
    # Sort events chronologically
    events_sorted = events.sort("event_timestamp")
    events_list = events_sorted.to_dicts()
    
    chain = []
    chain_started = False
    current_holder = None
    
    for event in events_list:
        # Skip events with missing addresses
        if event.get("from_address") is None or event.get("to_address") is None:
            continue
        
        if not chain_started:
            # Look for the starting event that matches start_pair.
            if event["from_address"] == start_pair[0] and event["to_address"] == start_pair[1]:
                chain_started = True
                chain.append({
                    "token_id": token_id,
                    "transfer_from": event["from_address"],
                    "transfer_to": event["to_address"],
                    "event_timestamp": event["event_timestamp"]
                })
                # Set the current holder to the receiver from the starting event.
                current_holder = event["to_address"]
                # If the starting event also matches the stop_pair, we are done.
                if event["from_address"] == stop_pair[0] and event["to_address"] == stop_pair[1]:
                    break
        else:
            # After chain has started, look for events where from_address equals the current holder.
            if event["from_address"] == current_holder:
                chain.append({
                    "token_id": token_id,
                    "transfer_from": event["from_address"],
                    "transfer_to": event["to_address"],
                    "event_timestamp": event["event_timestamp"]
                })
                current_holder = event["to_address"]
                # Stop once we reach the final event as specified by stop_pair.
                if event["from_address"] == stop_pair[0] and event["to_address"] == stop_pair[1]:
                    break
    return pl.DataFrame(chain)

# --- Example Usage ---

# For a given token, assume df_table1 (our summary table) has one row, for example:
# (The actual df_table1 likely has one row per token)
# Columns: token_id, seller_n-1_sale, buyer_n-1_sale, seller_n_sale, buyer_n_sale, etc.
# For illustration, we'll define them manually:

token_id = "1457"

# Define the start pair (should match an event where transfer_from == seller_n-1_sale and transfer_to == buyer_n-1_sale)
start_pair = (
    "0x29469395eaf6f95920e59f858042f0e28d98a20b",   # Replace with the actual seller_n-1_sale from df_table1
    "0x064980edd7d43abee781f49a4e31c06df05ecccb"    # Replace with the actual buyer_n-1_sale from df_table1
)
# Define the stop pair (the event that signals end of the chain)
stop_pair = (
    "0x064980edd7d43abee781f49a4e31c06df05ecccb",  # seller_n_sale
    "0x5f1ee29361206f1a129e808736f11598356c6031"   # buyer_n_sale
)

# Option 1: Assume df_transfer_events_pl already exists and has columns: 
# "nft_identifier", "from_address", "to_address", "event_timestamp"
# Ensure that the token_id column exists (rename if necessary)
df_transfer_events_pl = df_transfer_events_pl.with_columns(pl.col("nft_identifier").alias("token_id"))

# Filter the transfer events for the given token.
df_token_transfers = df_transfer_events_pl.filter(pl.col("token_id") == token_id)

# Optional: Ensure necessary columns are available (rename if necessary)
df_token_transfers = df_token_transfers.with_columns([
    pl.col("token_id"),
    pl.col("from_address"),
    pl.col("to_address"),
    pl.col("event_timestamp")
])

# Build the transfer chain for this token.
df_chain = build_transfer_chain(df_token_transfers, token_id, start_pair, stop_pair)

# Display the resulting transfer chain
print(df_chain)


shape: (6, 4)
┌──────────┬─────────────────────────────────┬─────────────────────────────────┬─────────────────┐
│ token_id ┆ transfer_from                   ┆ transfer_to                     ┆ event_timestamp │
│ ---      ┆ ---                             ┆ ---                             ┆ ---             │
│ str      ┆ str                             ┆ str                             ┆ i64             │
╞══════════╪═════════════════════════════════╪═════════════════════════════════╪═════════════════╡
│ 1457     ┆ 0x29469395eaf6f95920e59f858042… ┆ 0x064980edd7d43abee781f49a4e31… ┆ 1701325619      │
│ 1457     ┆ 0x064980edd7d43abee781f49a4e31… ┆ 0x29469395eaf6f95920e59f858042… ┆ 1701325967      │
│ 1457     ┆ 0x29469395eaf6f95920e59f858042… ┆ 0x064980edd7d43abee781f49a4e31… ┆ 1701417155      │
│ 1457     ┆ 0x064980edd7d43abee781f49a4e31… ┆ 0xaaa2da255df9ee74c7075bcb6d81… ┆ 1701417347      │
│ 1457     ┆ 0xaaa2da255df9ee74c7075bcb6d81… ┆ 0x064980edd7d43abee781f49a4e31… ┆ 1701665315    

In [ ]:
import polars as pl

# --- STEP 1: Define the helper function for building a transfer chain for one token ---
def build_transfer_chain(events: pl.DataFrame, token_id: str, start_pair: tuple, stop_pair: tuple) -> list:
    """
    Given a Polars DataFrame of transfer events for one token (sorted by event_timestamp),
    build a chain of transfers that starts at the event matching start_pair and continues 
    until an event matching stop_pair is encountered.
    
    Parameters:
      events: A Polars DataFrame containing transfer events for one token.
      token_id: The token identifier.
      start_pair: A tuple (start_from, start_to) representing the starting transfer (seller_n-1_sale, buyer_n-1_sale).
      stop_pair: A tuple (stop_from, stop_to) representing the final transfer (seller_n_sale, buyer_n_sale).
      
    Returns:
      A list of dictionaries with columns: token_id, transfer_from, transfer_to, event_timestamp.
      The list will be empty if no valid chain is found.
    """
    chain = []
    # Sort the events chronologically by event_timestamp
    events_sorted = events.sort("event_timestamp")
    events_list = events_sorted.to_dicts()  # iterate using a list of dictionaries
    
    chain_started = False
    current_holder = None  # This will eventually track the "to_address" from each accepted event
    
    for event in events_list:
        # Skip events with missing sender or receiver
        if event.get("from_address") is None or event.get("to_address") is None:
            continue

        # If the chain hasn't started, look for the starting event that exactly matches the start_pair.
        if not chain_started:
            if event["from_address"] == start_pair[0] and event["to_address"] == start_pair[1]:
                chain_started = True
                chain.append({
                    "token_id": token_id,
                    "transfer_from": event["from_address"],
                    "transfer_to": event["to_address"],
                    "event_timestamp": event["event_timestamp"]
                })
                current_holder = event["to_address"]
                # If the starting event already equals the stop_pair, then the chain ends here.
                if event["from_address"] == stop_pair[0] and event["to_address"] == stop_pair[1]:
                    break
        else:
            # Once the chain has started, only accept events where the sender matches the current holder.
            if event["from_address"] == current_holder:
                chain.append({
                    "token_id": token_id,
                    "transfer_from": event["from_address"],
                    "transfer_to": event["to_address"],
                    "event_timestamp": event["event_timestamp"]
                })
                # Update current holder
                current_holder = event["to_address"]
                # Check if we've reached the stop condition.
                if event["from_address"] == stop_pair[0] and event["to_address"] == stop_pair[1]:
                    break
    return chain

# --- STEP 2: Loop over the summary table (df_table1) and apply the chain walk ---
# df_table1 have these columns:
#   token_id, seller_n-1_sale, buyer_n-1_sale, seller_n_sale, buyer_n_sale, and possibly others.
#
# And the full transfer events are in df_transfer_events_pl, with:
#   nft_identifier (which we'll alias as token_id), from_address, to_address, event_timestamp.

# Ensure the transfer events DataFrame has a token_id column.
df_transfer_events_pl = df_transfer_events_pl.with_columns(
    pl.col("nft_identifier").alias("token_id")
)

all_chains = []  # This list will accumulate transfer chain records for all tokens.

# Convert df_table1 to a list of dictionaries for iteration.
table1_rows = df_table1.to_dicts()

for row in table1_rows:
    # Extract the token identifier and both starting and stopping addresses.
    token_id     = row.get("token_id")
    # For starting event, use seller_n-1_sale as the sender and buyer_n-1_sale as the receiver.
    start_pair = (row.get("seller_n-1_sale"), row.get("buyer_n-1_sale"))
    # For stopping event, use seller_n_sale as the sender and buyer_n_sale as the receiver.
    stop_pair  = (row.get("seller_n_sale"), row.get("buyer_n_sale"))
    
    # Skip if any of the needed values is missing.
    if token_id is None or None in start_pair or None in stop_pair:
        continue
    
    # Filter transfer events for this token.
    token_events = df_transfer_events_pl.filter(pl.col("token_id") == token_id)
    
    # If no events are found for this token, skip.
    if token_events.height == 0:
        continue
    
    # Build the transfer chain for this token.
    chain = build_transfer_chain(token_events, token_id, start_pair, stop_pair)
    
    # Add the chain to the overall list if it is nonempty.
    if chain:
        all_chains.extend(chain)

# --- STEP 3: Construct the final DataFrame from the accumulated chain records ---
if all_chains:
    df_table2 = pl.DataFrame(all_chains)
else:
    df_table2 = pl.DataFrame({
        "token_id": [],
        "transfer_from": [],
        "transfer_to": [],
        "event_timestamp": []
    })

# Optionally set display configuration to avoid truncation.
pl.Config.set_tbl_rows(1000)
pl.Config.set_tbl_cols(1000)

# Display the final DataFrame containing the transfer chains.
print(df_table2)


shape: (34_283, 4)
┌──────────┬─────────────────────────────────┬─────────────────────────────────┬─────────────────┐
│ token_id ┆ transfer_from                   ┆ transfer_to                     ┆ event_timestamp │
│ ---      ┆ ---                             ┆ ---                             ┆ ---             │
│ str      ┆ str                             ┆ str                             ┆ i64             │
╞══════════╪═════════════════════════════════╪═════════════════════════════════╪═════════════════╡
│ 1457     ┆ 0x29469395eaf6f95920e59f858042… ┆ 0x064980edd7d43abee781f49a4e31… ┆ 1701325619      │
│ 1457     ┆ 0x064980edd7d43abee781f49a4e31… ┆ 0x29469395eaf6f95920e59f858042… ┆ 1701325967      │
│ 1457     ┆ 0x29469395eaf6f95920e59f858042… ┆ 0x064980edd7d43abee781f49a4e31… ┆ 1701417155      │
│ 1457     ┆ 0x064980edd7d43abee781f49a4e31… ┆ 0xaaa2da255df9ee74c7075bcb6d81… ┆ 1701417347      │
│ 1457     ┆ 0xaaa2da255df9ee74c7075bcb6d81… ┆ 0x064980edd7d43abee781f49a4e31… ┆ 170166531

In [42]:
# Save df_table2 to a CSV file

csv_path = os.path.join(anaylsis_dir, 'df_table2.csv')
df_table2.write_csv(csv_path, include_header=True, separator=',')
print(f"DataFrame saved to {csv_path}")

DataFrame saved to C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\\df_table2.csv


# Table 3 NFT Characteristics

In [6]:


# --- Step 1: Load JSON data ---
json_path = r"C:\Emory\Research\NFT\Project\NFT_Details\full_nft_details.json"
with open(json_path, "r") as f:
    data = json.load(f)

# --- Step 2: Extract the traits in long format ---
# This yields one row per trait per NFT.
# The 'meta' parameter pulls NFT-level metadata (like token_id and rarity.rank)
df_traits_long = json_normalize(
    data['nfts'],
    record_path='traits',
    meta=['token_id', 'contract', 'collection', 'creator', ['rarity', 'rank'], 'name', 'description', 'image_url', 'metadata_url', 'opensea_url', 'updated_at', 'last_sale'],
    record_prefix='trait_'
)

# --- Step 3: Pivot traits to wide format ---
# Use token_id as the index and pivot "type" into columns with the "value" as content.
df_traits_wide = df_traits_long.pivot_table(
    index='token_id',
    columns='trait_type', 
    values='trait_value',
    aggfunc='first'  # In case of duplicate trait types, take the first encountered.
).reset_index()

# --- Step 4: Merge wide traits with NFT-level metadata ---
# Since metadata appears on each row in df_traits_long, drop duplicates to extract NFT-level info.
df_meta = df_traits_long.drop_duplicates(subset='token_id')[
    ['token_id', 'contract', 'collection', 'creator', 'rarity.rank', 'name', 'description', 'image_url', 'metadata_url', 'opensea_url', 'updated_at', 'last_sale']
]

# Now merge on "token_id"
df_final = pd.merge(df_meta, df_traits_wide, on='token_id', how='left')

# fill all blank cells with NaN
df_final = df_final.fillna(value=pd.NA)

# --- Step 5: Check and Save ---
print(df_final.head())
df_final.to_csv(r"C:\Emory\Research\NFT\Project\NFT_Details\full_nft_details_wide.csv", index=False)


C:\Users\Micheli\AppData\Local\Temp\ipykernel_44272\3028595208.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_final.fillna(value=pd.NA)


  token_id                                    contract         collection  \
0     3589  0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d  boredapeyachtclub   
1     3131  0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d  boredapeyachtclub   
2     9974  0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d  boredapeyachtclub   
3     9973  0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d  boredapeyachtclub   
4     9949  0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d  boredapeyachtclub   

                                      creator  rarity.rank  name description  \
0  0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03         6016  <NA>        <NA>   
1  0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03         9982  <NA>        <NA>   
2  0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03         1012  <NA>        <NA>   
3  0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03         6547  <NA>        <NA>   
4  0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03         2468  <NA>        <NA>   

                                           image_url  \


In [9]:
# Create df_table3 
# read csv file full_nft_details_wide.csv
df_table3 = pl.read_csv(r"C:\Emory\Research\NFT\Project\NFT_Details\full_nft_details_wide.csv")

# keep the following columns to df_table3
df_table3 = df_table3.select([
    pl.col("token_id"),
    pl.col("rarity.rank"),
    pl.col("Background"),
    pl.col("Clothes"),
    pl.col("Earring"),
    pl.col("Eyes"),
    pl.col("Fur"),
    pl.col("Hat"),
    pl.col("Mouth")
])

df_table3


token_id,rarity.rank,Background,Clothes,Earring,Eyes,Fur,Hat,Mouth
i64,i64,str,str,str,str,str,str,str
3589,6016,"""Gray""","""Bayc T Black""","""Silver Stud""","""3d""","""Brown""",null,"""Grin"""
3131,9982,"""Orange""",null,null,"""Sleepy""","""Dark Brown""",null,"""Bored Unshaven"""
9974,1012,"""Aquamarine""","""Rainbow Suspenders""","""Silver Hoop""","""Eyepatch""","""Dark Brown""","""Seaman's Hat""","""Tongue Out"""
9973,6547,"""Yellow""",null,null,"""Robot""","""Dmt""","""Girl's Hair Pink""","""Bored"""
9949,2468,"""Army Green""","""Guayabera""",null,"""3d""","""Tan""","""Seaman's Hat""","""Grin Gold Grill"""
…,…,…,…,…,…,…,…,…
4,3249,"""Blue""","""Navy Striped Tee""",null,"""Closed""","""Golden Brown""","""Party Hat 2""","""Phoneme L"""
3,7509,"""Purple""","""Bone Necklace""",null,"""Bored""","""Cheetah""",null,"""Tongue Out"""
2,7200,"""Aquamarine""",null,null,"""3d""","""Robot""","""Sea Captain's Hat""","""Bored Cigarette"""


In [43]:
# Save df_table3 to a CSV file
csv_path = os.path.join(anaylsis_dir, 'df_table3.csv')
df_table3.write_csv(csv_path, include_header=True, separator=',')

# Table 4 Buyer_n_Sale 

5296 unique 

In [ ]:
# buyer table consists of the following columns:
# buyer_address, active_period(total time range from first sale to 03/10/2025), total_value(sum all transaction values), 
# total_gasUsed(sum of all gasUsed), avg_gasPrice(average of all gasPrice), average_gasLimit(average of all gas) 

# Set up JSON directory and file pattern.
import os
import json
import datetime
import polars as pl

# Define the directory where JSON files are stored
json_dir_address = r"C:\Emory\Research\NFT\Project\Address_Data"
end_date = datetime.datetime(2025, 3, 10, tzinfo=datetime.timezone.utc)
end_timestamp = int(end_date.timestamp())  # Convert end date to Unix timestamp
df_table1  = pl.read_csv("df_table1.csv")  # Load df_table1 from CSV

# Assuming df_table1 is already loaded and contains the 'buyer_n_sale' column
# Extract unique buyer addresses from the 'buyer_n_sale' column of df_table1
buyer_addresses = df_table1.get_column("buyer_n_sale").unique().to_list()

# Initialize list to store metrics for each buyer
metrics = []

for address in buyer_addresses:
    filepath = os.path.join(json_dir_address, f"{address}.json")
    if not os.path.exists(filepath):
        print(f"Warning: JSON file not found for buyer address {address}. Skipping.")
        continue
    
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            data = json.load(file)
            transactions = data.get('transactions', [])
            
            if not transactions:
                print(f"No transactions found for {address}. Skipping.")
                continue
            
            # Initialize lists to collect transaction data
            values = []
            gas_used_list = []
            gas_prices = []
            gas_limits = []
            timestamps = []
            
            for tx in transactions:
                # Convert value from Wei to ETH and collect
                value = int(tx['value']) / 1e18 if tx['value'].strip() != '' else 0.0
                values.append(value)
                
                # Collect gas-related metrics
                gas_used_list.append(int(tx['gasUsed']))
                gas_prices.append(int(tx['gasPrice']))
                gas_limits.append(int(tx['gas']))
                
                # Collect timestamps
                timestamps.append(int(tx['timeStamp']))
            
            # Calculate active period in days
            if timestamps:
                first_tx_timestamp = min(timestamps)
                active_period_seconds = max(end_timestamp - first_tx_timestamp, 0)
                active_period_days = active_period_seconds / (24 * 3600)
            else:
                active_period_days = 0.0
            
            # Calculate totals and averages
            total_value = sum(values)
            total_gasUsed = sum(gas_used_list)
            avg_gasPrice = sum(gas_prices) / len(gas_prices) if gas_prices else 0.0
            avg_gasLimit = sum(gas_limits) / len(gas_limits) if gas_limits else 0.0
            
            # Append metrics for this buyer
            metrics.append({
                'buyer_n_address': address,
                'active_period': active_period_days,
                'total_value': total_value,
                'total_gasUsed': total_gasUsed,
                'avg_gasPrice': avg_gasPrice,
                'average_gasLimit': avg_gasLimit
            })
    
    except json.JSONDecodeError as e:
        print(f"JSON decoding error for {address}: {e}")
    except Exception as e:
        print(f"Error processing {address}: {e}")

# Create the buyer metrics DataFrame
df_table4 = pl.DataFrame(metrics)

# Display summary and save
print(f"✅ Successfully created buyer table with {df_table4.height} entries")
print(df_table4.head())

# Save to CSV (adjust path as needed)
df_table4.write_csv("df_table4.csv")


# Table N-2 Buyer

In [ ]:
# buyer table consists of the following columns:
# buyer_address, active_period(total time range from first sale to 03/10/2025), total_value(sum all transaction values), 
# total_gasUsed(sum of all gasUsed), avg_gasPrice(average of all gasPrice), average_gasLimit(average of all gas) 

# Set up JSON directory and file pattern.
import os
import json
import datetime
import polars as pl

# Define the directory where JSON files are stored
json_dir_address = r"C:\Emory\Research\NFT\Project\Address_Data"
end_date = datetime.datetime(2025, 3, 10, tzinfo=datetime.timezone.utc)
end_timestamp = int(end_date.timestamp())  # Convert end date to Unix timestamp
df_table1  = pl.read_csv("df_table1.csv")  # Load df_table1 from CSV

# Assuming df_table1 is already loaded and contains the 'buyer_n_sale' column
# Extract unique buyer addresses from the 'buyer_n_sale' column of df_table1
buyer_addresses = df_table1.get_column("buyer_n_sale").unique().to_list()

# Initialize list to store metrics for each buyer
metrics = []

for address in buyer_addresses:
    filepath = os.path.join(json_dir_address, f"{address}.json")
    if not os.path.exists(filepath):
        print(f"Warning: JSON file not found for buyer address {address}. Skipping.")
        continue
    
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            data = json.load(file)
            transactions = data.get('transactions', [])
            
            if not transactions:
                print(f"No transactions found for {address}. Skipping.")
                continue
            
            # Initialize lists to collect transaction data
            values = []
            gas_used_list = []
            gas_prices = []
            gas_limits = []
            timestamps = []
            
            for tx in transactions:
                # Convert value from Wei to ETH and collect
                value = int(tx['value']) / 1e18 if tx['value'].strip() != '' else 0.0
                values.append(value)
                
                # Collect gas-related metrics
                gas_used_list.append(int(tx['gasUsed']))
                gas_prices.append(int(tx['gasPrice']))
                gas_limits.append(int(tx['gas']))
                
                # Collect timestamps
                timestamps.append(int(tx['timeStamp']))
            
            # Calculate active period in days
            if timestamps:
                first_tx_timestamp = min(timestamps)
                active_period_seconds = max(end_timestamp - first_tx_timestamp, 0)
                active_period_days = active_period_seconds / (24 * 3600)
            else:
                active_period_days = 0.0
            
            # Calculate totals and averages
            total_value = sum(values)
            total_gasUsed = sum(gas_used_list)
            avg_gasPrice = sum(gas_prices) / len(gas_prices) if gas_prices else 0.0
            avg_gasLimit = sum(gas_limits) / len(gas_limits) if gas_limits else 0.0
            
            # Append metrics for this buyer
            metrics.append({
                'buyer_n_address': address,
                'active_period': active_period_days,
                'total_value': total_value,
                'total_gasUsed': total_gasUsed,
                'avg_gasPrice': avg_gasPrice,
                'average_gasLimit': avg_gasLimit
            })
    
    except json.JSONDecodeError as e:
        print(f"JSON decoding error for {address}: {e}")
    except Exception as e:
        print(f"Error processing {address}: {e}")

# Create the buyer metrics DataFrame
df_table4 = pl.DataFrame(metrics)

# Display summary and save
print(f"✅ Successfully created buyer table with {df_table4.height} entries")
print(df_table4.head())

# Save to CSV (adjust path as needed)
df_table4.write_csv("df_table4.csv")


# Table  Offer 

Aggregate per token_id, producing:

*   Total offers

*   Timestamps and prices of the latest (n), second-latest (n‑1), and earliest (1st) offers

*   Maker and taker of the latest offer

*   Offer event count from metadata

In [4]:
import os
import glob
import json
import pandas as pd
import numpy as np
from collections import defaultdict

# Load final sales data (df_table1) with robust token_id handling
sales_df = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table1.csv")
sales_df['token_id'] = sales_df['token_id'].astype(str).str.strip()
sales_dict = sales_df.set_index('token_id')[['time_n_sale', 'price_n_sale']].to_dict('index')

# Directory containing your offer JSON files
offer_json_dir = r"C:\Emory\Research\NFT\Project\NFT_Event_Offer"
offer_files = glob.glob(os.path.join(offer_json_dir, 'NFT_*_*.json'))

# Initialize aggregated summary
offers_summary = defaultdict(lambda: {
    'total_offers': 0,
    'offer_prices': [],
    'timestamps': [],
    'unique_makers': set(),
    'offers_above_sale': 0,
    'offers_below_sale': 0
})

# Stream-process each JSON file individually
for file in offer_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        token_id = str(data['metadata']['token_id']).strip()

        if token_id not in sales_dict:
            continue  # skip NFTs without a matching sale record

        final_price = sales_dict[token_id]['price_n_sale']
        final_sale_time = sales_dict[token_id]['time_n_sale']

        for event in data.get('events', []):
            offer_price = int(event['payment']['quantity']) / 1e18
            timestamp = int(event['event_timestamp'])
            maker = event.get('maker', None)

            summary = offers_summary[token_id]
            summary['total_offers'] += 1
            summary['offer_prices'].append(offer_price)
            summary['timestamps'].append(timestamp)
            if maker:
                summary['unique_makers'].add(maker)

            # Count offers above/below final sale price
            if offer_price > final_price:
                summary['offers_above_sale'] += 1
            elif offer_price < final_price:
                summary['offers_below_sale'] += 1

# After processing all files, compute robust aggregates
aggregated_data = []
for token_id, summary in offers_summary.items():
    prices = np.array(summary['offer_prices'])
    times = np.array(summary['timestamps'])
    sorted_times = np.sort(times)

    final_sale_time = sales_dict[token_id]['time_n_sale']
    final_price = sales_dict[token_id]['price_n_sale']

    # Residuals calculation
    residuals = prices - final_price
    time_residuals = final_sale_time - times  # seconds before sale

    aggregated_record = {
        'token_id': token_id,
        'total_offers': summary['total_offers'],
        'unique_makers_count': len(summary['unique_makers']),
        'mean_offer_price': np.mean(prices),
        'std_offer_price': np.std(prices),
        'median_offer_price': np.median(prices),
        'highest_offer': np.max(prices),
        'lowest_offer': np.min(prices),
        'mean_price_residual': np.mean(residuals),
        'std_price_residual': np.std(residuals),
        'mean_time_residual_hours': np.mean(time_residuals) / 3600,
        'std_time_residual_hours': np.std(time_residuals) / 3600,
        'offers_above_sale': summary['offers_above_sale'],
        'offers_below_sale': summary['offers_below_sale'],
        'duration_offer_activity_days': (sorted_times[-1] - sorted_times[0]) / (3600*24) if len(sorted_times) > 1 else 0,
        'time_since_last_offer_hours': (final_sale_time - sorted_times[-1]) / 3600,
        'offers_7d': np.sum(time_residuals <= 7 * 24 * 3600),
        'offers_14d': np.sum(time_residuals <= 14 * 24 * 3600),
        'offers_30d': np.sum(time_residuals <= 30 * 24 * 3600)
    }

    aggregated_data.append(aggregated_record)

# Save aggregated offer summary as CSV
offer_summary_df = pd.DataFrame(aggregated_data)
offer_summary_df.to_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table7.csv", index=False)

print(offer_summary_df.head())


  token_id  total_offers  unique_makers_count  mean_offer_price  \
0     1000           187                   13          0.009566   
1     1001          6598                   16         13.425134   
2     1002           187                   13          0.009501   
3     1003         14215                   23         11.846952   
4     1004           273                   17          5.888902   

   std_offer_price  median_offer_price  highest_offer  lowest_offer  \
0         0.006351              0.0069         0.0479  1.669000e-11   
1         3.530142             12.9600        22.0420  1.669000e-11   
2         0.006294              0.0069         0.0479  1.669000e-11   
3         3.070572             11.5500        21.8797  1.669000e-11   
4         8.667247              0.0100        22.2200  1.669000e-11   

   mean_price_residual  std_price_residual  mean_time_residual_hours  \
0            -0.490434            0.006351             -28316.814332   
1            -8.684866    

In [ ]:
import os
import glob
import json
from datetime import datetime
from collections import defaultdict

import pandas as pd
import numpy as np

# ─── PARAMETERS ────────────────────────────────────────────────────────────
OFFER_JSON_DIR = r"C:\Emory\Research\NFT\Project\NFT_Event_Offer"
OUTPUT_CSV     = r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\offers_monthly_summary.csv"

# ─── 1) COLLECT OFFERS INTO (token_id, year_month) BUCKETS ─────────────────
# Initialize container
offers_summary = defaultdict(lambda: {
    'prices':    [],
    'timestamps':[],
    'makers':    set()
})

# Scan all offer files
offer_files = glob.glob(os.path.join(OFFER_JSON_DIR, 'NFT_*_*.json'))
for filepath in offer_files:
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    token_id = str(data.get('metadata', {}).get('token_id', '')).strip()
    if not token_id:
        continue

    for ev in data.get('events', []):
        # Only item_offer orders
        if ev.get('event_type') != 'order' or ev.get('order_type') != 'item_offer':
            continue

        ts = ev.get('event_timestamp')
        if ts is None:
            continue

        # Convert Wei → ETH
        price = int(ev['payment']['quantity']) / 1e18
        # Year-month string
        ym = datetime.utcfromtimestamp(ts).strftime('%Y-%m')

        key = (token_id, ym)
        summary = offers_summary[key]
        summary['prices'].append(price)
        summary['timestamps'].append(ts)
        maker = ev.get('maker')
        if maker:
            summary['makers'].add(maker)

# ─── 2) COMPUTE MONTHLY AGGREGATES ──────────────────────────────────────────
records = []
for (token_id, ym), summary in offers_summary.items():
    prices = np.array(summary['prices'], dtype=float)
    times  = np.array(summary['timestamps'], dtype=int)

    rec = {
        'token_id':               token_id,
        'year_month':             ym,
        'total_offers':           len(prices),
        'unique_makers_count':    len(summary['makers']),
        'mean_offer_price':       prices.mean() if len(prices) else np.nan,
        'std_offer_price':        prices.std(ddof=0) if len(prices) else np.nan,
        'median_offer_price':     np.median(prices) if len(prices) else np.nan,
        'highest_offer':          prices.max() if len(prices) else np.nan,
        'lowest_offer':           prices.min() if len(prices) else np.nan,
        'duration_offer_days':    ((times.max() - times.min()) / 86400)
                                  if len(times) > 1 else 0
    }
    records.append(rec)

# ─── 3) SAVE TO CSV ─────────────────────────────────────────────────────────
out_df = pd.DataFrame(records)
out_df.to_csv(OUTPUT_CSV, index=False)
print(f"✔ Wrote {len(out_df)} rows of monthly offer summaries to:\n  {OUTPUT_CSV}")
print(out_df.head())


C:\Users\Micheli\AppData\Local\Temp\ipykernel_7344\810248385.py:44: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  ym = datetime.utcfromtimestamp(ts).strftime('%Y-%m')


✔ Wrote 97824 rows of monthly offer summaries to:
  C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\offers_monthly_summary.csv
  token_id year_month  total_offers  unique_makers_count  mean_offer_price  \
0        0    2025-03            15                    2      1.666133e+00   
1        0    2025-01            27                    2      1.944444e-02   
2        0    2024-12             3                    1      2.135333e-11   
3        0    2024-11             8                    1      1.100000e-02   
4        0    2024-10            54                    4      1.075926e-02   

   std_offer_price  median_offer_price  highest_offer  lowest_offer  \
0     4.237640e+00        3.500000e-03   1.250000e+01  3.500000e-03   
1     5.499719e-03        2.100000e-02   2.100000e-02  1.669000e-11   
2     2.608196e-12        2.001000e-11   2.500000e-11  1.905000e-11   
3     0.000000e+00        1.100000e-02   1.100000e-02  1.100000e-02   
4     1.137495e-03        1.100000e-0

# Panel Data

In [ ]:
import os
import glob
import json
import polars as pl
from datetime import datetime

# ========================================================================
# Part 1: Process Sales Events (Create Time-Based Panel Records)
# ========================================================================

def process_sales_events():
    json_dir = r"C:\Emory\Research\NFT\Project\NFT_Event_Sale\\"
    file_pattern = os.path.join(json_dir, 'NFT_*_*.json')
    
    sales_data = []
    
    for filepath in glob.glob(file_pattern):
        with open(filepath, 'r', encoding='utf-8') as file:
            try:
                data = json.load(file)
                
                # Skip NFTs with no sales events
                if not data.get('events'):
                    continue
                
                # Safely get metadata and token_id
                metadata = data.get('metadata', {})
                token_id = metadata.get('token_id')
                if not token_id:
                    continue
                
                for event in data['events']:
                    if event.get('event_type') != 'sale':
                        continue
                    
                    # Handle payment data safely
                    payment = event.get('payment') or {}
                    try:
                        amount_wei = int(payment.get('quantity', 0))
                    except (ValueError, TypeError):
                        amount_wei = 0
                        
                    amount_eth = amount_wei / 1e18 if amount_wei else None
                    
                    # Handle timestamp conversion
                    ts = event.get('event_timestamp')
                    year_month = None
                    if ts:
                        try:
                            dt = datetime.utcfromtimestamp(ts)
                            year_month = dt.strftime('%Y-%m')
                        except (ValueError, TypeError):
                            pass
                    
                    sales_data.append({
                        'token_id': token_id,
                        'year_month': year_month,
                        'sale_amount': amount_eth,
                        'seller': event.get('seller'),
                        'buyer': event.get('buyer'),
                        'event_timestamp': ts
                    })
                    
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON file: {filepath}")
            except Exception as e:
                print(f"Error processing {filepath}: {str(e)}")
    
    # Create DataFrame and clean up
    df_sales = pl.DataFrame(sales_data).unique()  # Remove potential duplicates
    
    # Filter valid sales records
    return df_sales.filter(
        pl.col('sale_amount').is_not_null() &
        pl.col('year_month').is_not_null() &
        pl.col('token_id').is_not_null()
    ).sort(['token_id', 'event_timestamp'])

# ========================================================================
# Part 2: Process NFT Traits (Static Features)
# ========================================================================

def process_nft_traits():
    json_path = r"C:\Emory\Research\NFT\Project\NFT_Details\full_nft_details.json"
    
    try:
        with open(json_path, "r") as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error loading traits file: {str(e)}")
        return pl.DataFrame()
    
    traits_data = []
    required_traits = ['Background', 'Clothes', 'Earring', 'Eyes', 'Fur', 'Hat', 'Mouth']
    
    for nft in data.get('nfts', []):
        try:
            token_id = nft.get('token_id')
            if not token_id:
                continue
                
            record = {
                'token_id': token_id,
                'rarity_rank': (nft.get('rarity') or {}).get('rank'),
                'collection': nft.get('collection'),
                'contract': nft.get('contract')
            }
            
            # Initialize all traits to None
            for trait in required_traits:
                record[trait] = None
                
            # Populate traits from NFT data
            for trait in nft.get('traits', []):
                trait_type = trait.get('type')
                if trait_type in required_traits:
                    record[trait_type] = trait.get('value')
            
            traits_data.append(record)
            
        except Exception as e:
            print(f"Error processing NFT {token_id}: {str(e)}")
    
    return pl.DataFrame(traits_data).unique()

# ========================================================================
# Part 3: Create Final Panel Dataset
# ========================================================================

def create_panel_dataset():
    try:
        df_sales = process_sales_events()
        if df_sales.is_empty():
            print("No valid sales data found")
            return pl.DataFrame()
            
        df_traits = process_nft_traits()
        if df_traits.is_empty():
            print("No trait data found - using sales data only")
            return df_sales
            
        return df_sales.join(
            df_traits,
            on='token_id',
            how='left'
        ).select([
            'token_id', 'year_month', 'sale_amount',
            'seller', 'buyer', 'rarity_rank',
            'Background', 'Clothes', 'Earring',
            'Eyes', 'Fur', 'Hat', 'Mouth'
        ])
        
    except Exception as e:
        print(f"Failed to create panel dataset: {str(e)}")
        return pl.DataFrame()

# ========================================================================
# Execution and Validation
# ========================================================================

if __name__ == '__main__':
    panel_data = create_panel_dataset()
    
    if not panel_data.is_empty():
        print("Final Panel Dataset Schema:")
        print(panel_data.schema)
        print("\nSample Data:")
        print(panel_data.head(3))
        panel_data.write_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\nft_panel_dataset_full.csv")
        print("\nSaved panel dataset to CSV")
    else:
        print("No data to save")

C:\Users\Micheli\AppData\Local\Temp\ipykernel_28412\1436533886.py:50: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(ts)


Final Panel Dataset Schema:
Schema({'token_id': String, 'year_month': String, 'sale_amount': Float64, 'seller': String, 'buyer': String, 'rarity_rank': Int64, 'Background': String, 'Clothes': String, 'Earring': String, 'Eyes': String, 'Fur': String, 'Hat': String, 'Mouth': String})

Sample Data:
shape: (3, 13)
┌──────────┬────────────┬────────────┬────────────┬───┬───────────┬───────┬────────────┬───────────┐
│ token_id ┆ year_month ┆ sale_amoun ┆ seller     ┆ … ┆ Eyes      ┆ Fur   ┆ Hat        ┆ Mouth     │
│ ---      ┆ ---        ┆ t          ┆ ---        ┆   ┆ ---       ┆ ---   ┆ ---        ┆ ---       │
│ str      ┆ str        ┆ ---        ┆ str        ┆   ┆ str       ┆ str   ┆ str        ┆ str       │
│          ┆            ┆ f64        ┆            ┆   ┆           ┆       ┆            ┆           │
╞══════════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════╪════════════╪═══════════╡
│ 10       ┆ 2021-11    ┆ 80.0       ┆ 0x893a3701 ┆ … ┆ Eyepatch  ┆ Dmt   ┆ Bayc H

# Panel Data 

This outputs n-2 sales to avoid data leakage for Fixed Effect to go into PartB 

In [ ]:
import os
import glob
import json
import polars as pl
from datetime import datetime

# ========================================================================
# Part 1: Process Sales Events (Create Time-Based Panel Records)
# ========================================================================

def process_sales_events():
    json_dir = r"C:\Emory\Research\NFT\Project\NFT_Event_Sale\\"
    file_pattern = os.path.join(json_dir, 'NFT_*_*.json')
    
    sales_data = []
    
    for filepath in glob.glob(file_pattern):
        with open(filepath, 'r', encoding='utf-8') as file:
            try:
                data = json.load(file)
                
                if not data.get('events'):
                    continue
                
                metadata = data.get('metadata', {})
                token_id = metadata.get('token_id')
                if not token_id:
                    continue
                
                for event in data['events']:
                    if event.get('event_type') != 'sale':
                        continue
                    
                    payment = event.get('payment') or {}
                    try:
                        amount_wei = int(payment.get('quantity', 0))
                    except (ValueError, TypeError):
                        amount_wei = 0
                        
                    amount_eth = amount_wei / 1e18 if amount_wei else None
                    
                    ts = event.get('event_timestamp')
                    year_month = None
                    if ts:
                        try:
                            dt = datetime.utcfromtimestamp(ts)
                            year_month = dt.strftime('%Y-%m')
                        except (ValueError, TypeError):
                            pass
                    
                    sales_data.append({
                        'token_id': token_id,
                        'year_month': year_month,
                        'sale_amount': amount_eth,
                        'seller': event.get('seller'),
                        'buyer': event.get('buyer'),
                        'event_timestamp': ts
                    })
                    
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON file: {filepath}")
            except Exception as e:
                print(f"Error processing {filepath}: {str(e)}")
    
    df_sales = pl.DataFrame(sales_data).unique()
    
    # Modified section: Exclude last 2 sales per NFT
    return (
        df_sales
        .filter(
            pl.col('sale_amount').is_not_null() &
            pl.col('year_month').is_not_null() &
            pl.col('token_id').is_not_null()
        )
        .sort(['token_id', 'event_timestamp'])
        # Add reverse row count per NFT
        .with_columns(
            pl.arange(0, pl.count()).reverse().over('token_id').alias('reverse_row_num')
        )
        # Filter out last 2 rows per NFT group
        .filter(pl.col('reverse_row_num') >= 2)
        .drop('reverse_row_num')
        .sort(['token_id', 'event_timestamp'])
    )

# ========================================================================
# Part 2: Process NFT Traits (Static Features)
# ========================================================================

def process_nft_traits():
    json_path = r"C:\Emory\Research\NFT\Project\NFT_Details\full_nft_details.json"
    
    try:
        with open(json_path, "r") as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error loading traits file: {str(e)}")
        return pl.DataFrame()
    
    traits_data = []
    required_traits = ['Background', 'Clothes', 'Earring', 'Eyes', 'Fur', 'Hat', 'Mouth']
    
    for nft in data.get('nfts', []):
        try:
            token_id = nft.get('token_id')
            if not token_id:
                continue
                
            record = {
                'token_id': token_id,
                'rarity_rank': (nft.get('rarity') or {}).get('rank'),
                'collection': nft.get('collection'),
                'contract': nft.get('contract')
            }
            
            # Initialize all traits to None
            for trait in required_traits:
                record[trait] = None
                
            # Populate traits from NFT data
            for trait in nft.get('traits', []):
                trait_type = trait.get('type')
                if trait_type in required_traits:
                    record[trait_type] = trait.get('value')
            
            traits_data.append(record)
            
        except Exception as e:
            print(f"Error processing NFT {token_id}: {str(e)}")
    
    return pl.DataFrame(traits_data).unique()

# ========================================================================
# Part 3: Create Final Panel Dataset
# ========================================================================

def create_panel_dataset():
    try:
        df_sales = process_sales_events()
        if df_sales.is_empty():
            print("No valid sales data found")
            return pl.DataFrame()
            
        df_traits = process_nft_traits()
        if df_traits.is_empty():
            print("No trait data found - using sales data only")
            return df_sales
            
        return df_sales.join(
            df_traits,
            on='token_id',
            how='left'
        ).select([
            'token_id', 'year_month', 'sale_amount',
            'seller', 'buyer', 'rarity_rank',
            'Background', 'Clothes', 'Earring',
            'Eyes', 'Fur', 'Hat', 'Mouth'
        ])
        
    except Exception as e:
        print(f"Failed to create panel dataset: {str(e)}")
        return pl.DataFrame()

# ========================================================================
# Execution and Validation
# ========================================================================

if __name__ == '__main__':
    panel_data = create_panel_dataset()
    
    if not panel_data.is_empty():
        print("Final Panel Dataset Schema:")
        print(panel_data.schema)
        print("\nSample Data:")
        print(panel_data.head(3))
        panel_data.write_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\nft_panel_dataset.csv")
        print("\nSaved panel dataset to CSV")
    else:
        print("No data to save")

C:\Users\Micheli\AppData\Local\Temp\ipykernel_29332\3396269015.py:46: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(ts)
C:\Users\Micheli\AppData\Local\Temp\ipykernel_29332\3396269015.py:78: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.arange(0, pl.count()).reverse().over('token_id').alias('reverse_row_num')


Final Panel Dataset Schema:
Schema({'token_id': String, 'year_month': String, 'sale_amount': Float64, 'seller': String, 'buyer': String, 'rarity_rank': Int64, 'Background': String, 'Clothes': String, 'Earring': String, 'Eyes': String, 'Fur': String, 'Hat': String, 'Mouth': String})

Sample Data:
shape: (3, 13)
┌──────────┬────────────┬────────────┬────────────┬───┬───────────┬───────┬────────────┬───────────┐
│ token_id ┆ year_month ┆ sale_amoun ┆ seller     ┆ … ┆ Eyes      ┆ Fur   ┆ Hat        ┆ Mouth     │
│ ---      ┆ ---        ┆ t          ┆ ---        ┆   ┆ ---       ┆ ---   ┆ ---        ┆ ---       │
│ str      ┆ str        ┆ ---        ┆ str        ┆   ┆ str       ┆ str   ┆ str        ┆ str       │
│          ┆            ┆ f64        ┆            ┆   ┆           ┆       ┆            ┆           │
╞══════════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════╪════════════╪═══════════╡
│ 100      ┆ 2021-09    ┆ 41.14      ┆ 0x35f0686c ┆ … ┆ Wide Eyed ┆ Dark  ┆ Party 

# Sales per month data merging for panel 

In [ ]:
import os
import glob
import json
from datetime import datetime
from collections import defaultdict

import pandas as pd
import numpy as np

# ─── PARAMETERS ────────────────────────────────────────────────────────────
OFFER_JSON_DIR = r"C:\Emory\Research\NFT\Project\NFT_Event_Offer"
OUTPUT_CSV     = r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\offers_monthly_summary.csv"

# ─── 1) COLLECT OFFERS INTO (token_id, year_month) BUCKETS ─────────────────
# Initialize container
offers_summary = defaultdict(lambda: {
    'prices':    [],
    'timestamps':[],
    'makers':    set()
})

# Scan all offer files
offer_files = glob.glob(os.path.join(OFFER_JSON_DIR, 'NFT_*_*.json'))
for filepath in offer_files:
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    token_id = str(data.get('metadata', {}).get('token_id', '')).strip()
    if not token_id:
        continue

    for ev in data.get('events', []):
        # Only item_offer orders
        if ev.get('event_type') != 'order' or ev.get('order_type') != 'item_offer':
            continue

        ts = ev.get('event_timestamp')
        if ts is None:
            continue

        # Convert Wei → ETH
        price = int(ev['payment']['quantity']) / 1e18
        # Year-month string
        ym = datetime.utcfromtimestamp(ts).strftime('%Y-%m')

        key = (token_id, ym)
        summary = offers_summary[key]
        summary['prices'].append(price)
        summary['timestamps'].append(ts)
        maker = ev.get('maker')
        if maker:
            summary['makers'].add(maker)

# ─── 2) COMPUTE MONTHLY AGGREGATES ──────────────────────────────────────────
records = []
for (token_id, ym), summary in offers_summary.items():
    prices = np.array(summary['prices'], dtype=float)
    times  = np.array(summary['timestamps'], dtype=int)

    rec = {
        'token_id':               token_id,
        'year_month':             ym,
        'total_offers':           len(prices),
        'unique_makers_count':    len(summary['makers']),
        'mean_offer_price':       prices.mean() if len(prices) else np.nan,
        'std_offer_price':        prices.std(ddof=0) if len(prices) else np.nan,
        'median_offer_price':     np.median(prices) if len(prices) else np.nan,
        'highest_offer':          prices.max() if len(prices) else np.nan,
        'lowest_offer':           prices.min() if len(prices) else np.nan,
        'duration_offer_days':    ((times.max() - times.min()) / 86400)
                                  if len(times) > 1 else 0
    }
    records.append(rec)

# ─── 3) SAVE TO CSV ─────────────────────────────────────────────────────────
out_df = pd.DataFrame(records)
out_df.to_csv(OUTPUT_CSV, index=False)
print(f"✔ Wrote {len(out_df)} rows of monthly offer summaries to:\n  {OUTPUT_CSV}")
print(out_df.head())


C:\Users\Micheli\AppData\Local\Temp\ipykernel_7344\810248385.py:44: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  ym = datetime.utcfromtimestamp(ts).strftime('%Y-%m')


✔ Wrote 97824 rows of monthly offer summaries to:
  C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\offers_monthly_summary.csv
  token_id year_month  total_offers  unique_makers_count  mean_offer_price  \
0        0    2025-03            15                    2      1.666133e+00   
1        0    2025-01            27                    2      1.944444e-02   
2        0    2024-12             3                    1      2.135333e-11   
3        0    2024-11             8                    1      1.100000e-02   
4        0    2024-10            54                    4      1.075926e-02   

   std_offer_price  median_offer_price  highest_offer  lowest_offer  \
0     4.237640e+00        3.500000e-03   1.250000e+01  3.500000e-03   
1     5.499719e-03        2.100000e-02   2.100000e-02  1.669000e-11   
2     2.608196e-12        2.001000e-11   2.500000e-11  1.905000e-11   
3     0.000000e+00        1.100000e-02   1.100000e-02  1.100000e-02   
4     1.137495e-03        1.100000e-0

# Generates a list of exact time for calculation 

In [ ]:
import duckdb

# ─── CONFIG ────────────────────────────────────────────────────────────────
SALES_JSON_DIR = r'C:\Emory\Research\NFT\Project\NFT_Event_Sale\NFT_*_*.json'
OUTPUT_CSV     = r'C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\sales_simple.csv'

# ─── 1) CONNECT ─────────────────────────────────────────────────────────────
con = duckdb.connect(database=':memory:')

# ─── 2) EXTRACT token_id + event_timestamp FOR EVERY SALE ───────────────────
con.execute(f"""
CREATE OR REPLACE TABLE sales_simple AS
SELECT
  doc.metadata.token_id::VARCHAR           AS token_id,
  CAST(event.event_timestamp AS BIGINT)    AS event_timestamp
FROM read_json_auto(
       '{SALES_JSON_DIR}',
       maximum_object_size := 50000000     -- bump per-file JSON limit to 50 MB
     ) AS doc
CROSS JOIN UNNEST(doc.events)            AS t(event)
WHERE event.event_type = 'sale'
;
""")

# ─── 3) EXPORT TO CSV ────────────────────────────────────────────────────────
con.execute(f"""
COPY sales_simple
TO '{OUTPUT_CSV}'
WITH (HEADER, NULL 'NULL');
""")

print("✅ Done. sales_simple written to:", OUTPUT_CSV)


# Panel Data for OLS and Fixed Effect Models

In [71]:
import os 
import pandas as pd
import polars as pl
import numpy as np 

# read table 
df_table_token_label = pl.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\sales_simple.csv")
df_offer_monthly = pl.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\offers_monthly_summary.csv")

print(len(df_offer_monthly))

# df_table_token_label token_id list 
token_label = df_table_token_label.get_column("token_id").unique().to_list()
print(len(token_label))

# filter out offers that are not in token_label 
df_offer_monthly = df_offer_monthly.filter(pl.col("token_id").is_in(token_label))
print(len(df_offer_monthly))

# convert df_table_token_label['event_timestamp'] to datetime
df_table_token_label = df_table_token_label.with_columns(
    pl.from_epoch("event_timestamp").alias("datetime")
)

# do a new table that basically does the following: 
# for each year-month-day, use the day/(total days in the month) for example january has 31 days, febuary has 28 days,
# so for january 1st, it will be 1/31, for febuary 1st, it will be 1/28, and so on.
# the new coulmn will be named "day_ratio"
df_table_token_label = df_table_token_label.with_columns(
    # Get the day of the month
    pl.col("datetime").dt.day().alias("day"),
    # Get the last day of the month (which equals the number of days in that month)
    pl.col("datetime").dt.month_end().dt.day().alias("days_in_month")
)

# Now calculate the ratio
df_table_token_label = df_table_token_label.with_columns(
    (pl.col("day") / pl.col("days_in_month")).alias("day_ratio")
)
# see some rows 
df_table_token_label.head(20)

# Add a year_month column for further filtering/joining
df_table_token_label = df_table_token_label.with_columns(
    pl.col("datetime").dt.strftime("%Y-%m").alias("year_month")
)

df_table_token_label = df_table_token_label.with_columns(
    pl.col("year_month")
      .str.strptime(pl.Date, format="%Y-%m") # Parse string to Date (defaults to 1st day)
      .dt.timestamp(time_unit="us")          # Convert Date to Unix timestamp (seconds)
      .alias("year_month_unix_start")       # Name the new column
)

# Now calculate the ratio
df_table_token_label = df_table_token_label.with_columns(
    (pl.col("event_timestamp") / pl.col("year_month_unix_start")).alias("unix_ratio")
)

# display the following row token_id = 1041 year_month =2025-01
df_table_token_label.filter(
    (pl.col("token_id") == 1041) & (pl.col("year_month") == "2025-01")
).head(20)


# now we caculates for each token_id with respect to year-month 
# times each column in the df_offer_monthly table and merge it to df_table_token_label table
# instead of merging it. read every rows of the following columns (total_offers, mean_offer_price, std_offer_price, highest_offer, lowest_offer)
# that will be a for loop 

# convert event_timestamp in df_table_token_label into year_month
df_table_token_label = df_table_token_label.with_columns(
    pl.col("datetime").dt.strftime("%Y-%m").alias("year_month")
)
# see some rows
df_table_token_label.head(5)

# columns in df_offer_monthly table
offer_cols_to_use = [
    "token_id",
    "year_month",
    "total_offers",
    "mean_offer_price",
    "std_offer_price",
    "highest_offer",
    "lowest_offer",
    "duration_offer_days"
]

df_merged = df_table_token_label.join(
    df_offer_monthly,
    on=["token_id", "year_month"],
    how="left" # Keep all rows from sales, add offer data where it matches
)

print(f"Length after join: {len(df_merged)}")
print("Columns after join:", df_merged.columns)

# number of not null values in each column
print(df_merged.select([pl.col(col).is_not_null().sum().alias(f"{col}_not_null") for col in offer_cols_to_use]))

# stats colums 

stats_cols = [
    "total_offers",
    "mean_offer_price",
    "std_offer_price",
    "highest_offer",
    "lowest_offer",
    "duration_offer_days"
]


# Create expressions for the weighted calculations
weighted_cols_exprs_overwrite = [
    (pl.col(col) * pl.col("unix_ratio")).alias(col) for col in stats_cols
]

df_final = df_merged.with_columns(weighted_cols_exprs_overwrite)



# columns to keep 
keep_cols = [
    "token_id",
    "year_month",
    "event_timestamp",
    "day_ratio",
    "unix_ratio",
    "total_offers",
    "mean_offer_price",
    "std_offer_price",
    "highest_offer",
    "lowest_offer",
    "duration_offer_days"
    
]

df_final = df_final.select(keep_cols)
print(f"Final DataFrame shape: {df_final.shape}")

# drop all na rows 
df_final = df_final.drop_nulls()
print(f"Final DataFrame shape after dropping nulls: {df_final.shape}")


print(f"Final DataFrame shape after merging with df_table2: {df_final.shape}")
print("Columns after merge:", df_final.columns)
print(df_final.head(5))

df_panel = pl.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\nft_panel_dataset.csv")

# merge df_final and df_panel
df_final = df_panel.join(
    df_final,
    on="token_id",
    how="left" # Keep all rows from sales, add offer data where it matches
)
df_final = df_final.drop_nulls()
print(f"Final DataFrame shape after merging with df_panel: {df_final.shape}")

# drop duplicates based on token_id and event_timestamp
df_final = df_final.unique(subset=["token_id", "event_timestamp"])
print(f"Final DataFrame shape after dropping duplicates: {df_final.shape}")


# save the final table
df_final.write_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\panel_for_var2.csv")

97824
9254
89833
Length after join: 57936
Columns after join: ['token_id', 'event_timestamp', 'datetime', 'day', 'days_in_month', 'day_ratio', 'year_month', 'year_month_unix_start', 'unix_ratio', 'total_offers', 'unique_makers_count', 'mean_offer_price', 'std_offer_price', 'median_offer_price', 'highest_offer', 'lowest_offer', 'duration_offer_days']
shape: (1, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ token_id_n ┆ year_month ┆ total_offe ┆ mean_offe ┆ std_offer ┆ highest_o ┆ lowest_of ┆ duration_ │
│ ot_null    ┆ _not_null  ┆ rs_not_nul ┆ r_price_n ┆ _price_no ┆ ffer_not_ ┆ fer_not_n ┆ offer_day │
│ ---        ┆ ---        ┆ l          ┆ ot_null   ┆ t_null    ┆ null      ┆ ull       ┆ s_not_nul │
│ u32        ┆ u32        ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ l         │
│            ┆            ┆ u32        ┆ u32       ┆ u32       ┆ u32       ┆ u32       ┆ ---       │
│            ┆            ┆ 

In [26]:

# read panel table 

df_panel = pl.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\nft_panel_dataset.csv")

df_panel.shape

(40542, 13)

# Panel Data for Last two Transactions Model2.Updated

In [ ]:
import pandas as pd

# 1. Load your original table
df = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table1.csv")

# 2. Define how to map each “time_of_sale” to its columns
mappings = [
    {
        "time_of_sale": "time_n_sale",
        "time_col":     "time_n_sale",
        "price_col":    "price_n_sale",
        "buyer_col":    "buyer_n_sale",
        "seller_col":   "seller_n_sale"
    },
    {
        "time_of_sale": "time_n-1_sale",
        "time_col":     "time_n-1_sale",
        "price_col":    "price_n-1_sale",
        "buyer_col":    "buyer_n-1_sale",
        "seller_col":   "seller_n-1_sale"
    }
]

# 3. Build the new long‐form rows
rows = []
for _, r in df.iterrows():
    for m in mappings:
        # Safely handle missing timestamps
        ts_val = r[m["time_col"]]
        if pd.notnull(ts_val):
            sale_time = pd.to_datetime(ts_val, unit="s").strftime("%Y-%m")
        else:
            sale_time = None

        rows.append({
            "token_id":     r["token_id"],
            "time_of_sale": m["time_of_sale"],
            "sale_time":    sale_time,
            "price":        r[m["price_col"]],
            "buyer":        r[m["buyer_col"]],
            "seller":       r[m["seller_col"]],
            "event_count":  r["event_count"]
        })

long_df = pd.DataFrame(rows)

# 4. reorder columns
long_df = long_df[[
    "token_id", "time_of_sale", "sale_time", "price",
    "buyer", "seller", "event_count"
]]

# 5. Inspect
print(long_df.head(4))

long_df.to_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table1_long.csv", index=False)

   token_id   time_of_sale sale_time   price  \
0      1457    time_n_sale   2023-12  29.910   
1      1457  time_n-1_sale   2023-11  29.140   
2      1810    time_n_sale   2024-12  17.799   
3      1810  time_n-1_sale   2024-12  19.960   

                                        buyer  \
0  0x5f1ee29361206f1a129e808736f11598356c6031   
1  0x064980edd7d43abee781f49a4e31c06df05ecccb   
2  0x0a60cf261abef4fcd00b549f63711d0f5b9d0da4   
3  0x29469395eaf6f95920e59f858042f0e28d98a20b   

                                       seller  event_count  
0  0x064980edd7d43abee781f49a4e31c06df05ecccb           23  
1  0x29469395eaf6f95920e59f858042f0e28d98a20b           23  
2  0xb65b4fd4dcb639b400fbd3d0ee5d272b5e49c2a8           20  
3  0x4fd5065f59643e343272db1ebcdddee194cae358           20  


### Rename columns and first initial merging

In [13]:
df_table1_long = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table1_long.csv")
# round the price column to 2 decimal places
df_table1_long['price'] = df_table1_long['price'].round(2)

df_table_offer = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\offers_monthly_summary.csv")

df_buyer_n = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table4.csv")
df_seller_n = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table5.csv")
df_buyer_n_1 = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table6.csv")
df_seller_n_1 = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table7.csv")

df_buyer_n.rename(columns={'transaction_count':'buyer_n_tscount',
                          'active_period':'buyer_n_act_period',
                          'total_value':'buyer_n_total_value',
                          'total_gasUsed':'buyer_n_total_gasUsed',
                          'avg_gasPrice':'buyer_n_avg_gasPrice',
                          'avg_gasLimit':'buyer_n_avg_gasLimit',
                          'rolling_avg_value_last10':'buyer_n_rolling_avg_value_last10',
                          'rolling_std_value_last10':'buyer_n_rolling_std_value_last10'}, inplace=True)
# Ensure 'time_of_sale' exists before filling
if 'time_of_sale' not in df_buyer_n.columns:
    df_buyer_n['time_of_sale'] = 'time_n_sale'
else:
    df_buyer_n['time_of_sale'] = df_buyer_n['time_of_sale'].fillna('time_n_sale')

df_seller_n.rename(columns={'transaction_count':'seller_n_tscount',
                          'active_period':'seller_n_act_period',
                          'total_value':'seller_n_total_value',
                          'total_gasUsed':'seller_n_total_gasUsed',
                          'avg_gasPrice':'seller_n_avg_gasPrice',
                          'avg_gasLimit':'seller_n_avg_gasLimit',
                          'rolling_avg_value_last10':'seller_n_rolling_avg_value_last10',
                          'rolling_std_value_last10':'seller_n_rolling_std_value_last10'}, inplace=True)
if 'time_of_sale' not in df_seller_n.columns:
    df_seller_n['time_of_sale'] = 'time_n_sale'
else:
    df_seller_n['time_of_sale'] = df_seller_n['time_of_sale'].fillna('time_n_sale')

df_buyer_n_1.rename(columns={'transaction_count':'buyer_n_1_tscount',
                          'active_period':'buyer_n_1_act_period',
                          'total_value':'buyer_n_1_total_value',
                          'total_gasUsed':'buyer_n_1_total_gasUsed',
                          'avg_gasPrice':'buyer_n_1_avg_gasPrice',
                          'avg_gasLimit':'buyer_n_1_avg_gasLimit',
                          'rolling_avg_value_last10':'buyer_n_1_rolling_avg_value_last10',
                          'rolling_std_value_last10':'buyer_n_1_rolling_std_value_last10'}, inplace=True)
if 'time_of_sale' not in df_buyer_n_1.columns:
    df_buyer_n_1['time_of_sale'] = 'time_n-1_sale'
else:
    df_buyer_n_1['time_of_sale'] = df_buyer_n_1['time_of_sale'].fillna('time_n-1_sale')

df_seller_n_1.rename(columns={'transaction_count':'seller_n_1_tscount',
                          'active_period':'seller_n_1_act_period',
                          'total_value':'seller_n_1_total_value',
                          'total_gasUsed':'seller_n_1_total_gasUsed',
                          'avg_gasPrice':'seller_n_1_avg_gasPrice',
                          'avg_gasLimit':'seller_n_1_avg_gasLimit',
                          'rolling_avg_value_last10':'seller_n_1_rolling_avg_value_last10',
                          'rolling_std_value_last10':'seller_n_1_rolling_std_value_last10'}, inplace=True)
if 'time_of_sale' not in df_seller_n_1.columns:
    df_seller_n_1['time_of_sale'] = 'time_n-1_sale'
else:
    df_seller_n_1['time_of_sale'] = df_seller_n_1['time_of_sale'].fillna('time_n-1_sale')

# merge 
df = pd.merge(df_table1_long, df_buyer_n, left_on=['buyer','time_of_sale'],right_on=['buyer_n_address','time_of_sale'], how='left')
df = pd.merge(df, df_seller_n, left_on=['seller','time_of_sale'],right_on=['seller_n_address','time_of_sale'], how='left')
df = pd.merge(df, df_buyer_n_1, left_on=['buyer','time_of_sale'],right_on=['buyer_n-1_address','time_of_sale'], how='left')
df = pd.merge(df, df_seller_n_1, left_on=['seller','time_of_sale'],right_on=['seller_n-1_address','time_of_sale'], how='left')

In [15]:
df.to_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\dfxxx.csv", index=False)

### Consolidated_buyers
fixing the empty rows

In [7]:
import pandas as pd
import numpy as np

# Load the long format sales data
df_table1_long = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table1_long.csv")
df_table1_long['price'] = df_table1_long['price'].round(2)

# Load the network stats data
df_buyer_n_stats = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table4.csv")
df_seller_n_stats = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table5.csv")
df_buyer_n_1_stats = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table6.csv")
df_seller_n_1_stats = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table7.csv")

# --- Rename columns to generic names BEFORE merging ---
# Define the target generic column names
generic_cols = {
    'transaction_count': 'tscount',
    'active_period': 'act_period',
    'total_value': 'total_value',
    'total_gasUsed': 'total_gasUsed',
    'avg_gasPrice': 'avg_gasPrice',
    'avg_gasLimit': 'avg_gasLimit',
    'rolling_avg_value_last10': 'rolling_avg_value_last10',
    'rolling_std_value_last10': 'rolling_std_value_last10'
}

# Rename buyer_n stats
df_buyer_n_stats = df_buyer_n_stats.rename(columns=generic_cols)
df_buyer_n_stats = df_buyer_n_stats.rename(columns={'buyer_n_address': 'address'}) # Use 'address' for joining

# Rename seller_n stats
df_seller_n_stats = df_seller_n_stats.rename(columns=generic_cols)
df_seller_n_stats = df_seller_n_stats.rename(columns={'seller_n_address': 'address'})

# Rename buyer_n_1 stats
df_buyer_n_1_stats = df_buyer_n_1_stats.rename(columns=generic_cols)
df_buyer_n_1_stats = df_buyer_n_1_stats.rename(columns={'buyer_n-1_address': 'address'})

# Rename seller_n_1 stats
df_seller_n_1_stats = df_seller_n_1_stats.rename(columns=generic_cols)
df_seller_n_1_stats = df_seller_n_1_stats.rename(columns={'seller_n-1_address': 'address'})

# --- Merge based on time_of_sale ---

# Merge for time_n_sale
df_n = df_table1_long[df_table1_long['time_of_sale'] == 'time_n_sale'].copy()
df_n = pd.merge(df_n, df_buyer_n_stats, left_on='buyer', right_on='address', how='left', suffixes=('', '_buyer_stats'))
df_n = pd.merge(df_n, df_seller_n_stats, left_on='seller', right_on='address', how='left', suffixes=('', '_seller_stats'))

# Rename columns specifically for buyer and seller for df_n
buyer_rename_n = {col: f'buyer_{col}' for col in generic_cols.values()}
seller_rename_n = {col: f'seller_{col}' for col in generic_cols.values()}
df_n = df_n.rename(columns=buyer_rename_n)
df_n = df_n.rename(columns=seller_rename_n)


# Merge for time_n-1_sale
df_n_1 = df_table1_long[df_table1_long['time_of_sale'] == 'time_n-1_sale'].copy()
df_n_1 = pd.merge(df_n_1, df_buyer_n_1_stats, left_on='buyer', right_on='address', how='left', suffixes=('', '_buyer_stats'))
df_n_1 = pd.merge(df_n_1, df_seller_n_1_stats, left_on='seller', right_on='address', how='left', suffixes=('', '_seller_stats'))

# Rename columns specifically for buyer and seller for df_n_1
buyer_rename_n_1 = {col: f'buyer_{col}' for col in generic_cols.values()}
seller_rename_n_1 = {col: f'seller_{col}' for col in generic_cols.values()}
df_n_1 = df_n_1.rename(columns=buyer_rename_n_1)
df_n_1 = df_n_1.rename(columns=seller_rename_n_1)

# --- Concatenate the results ---
df_final = pd.concat([df_n, df_n_1], ignore_index=True, sort=False)

# --- Clean up columns ---
# Define final columns to keep (adjust as needed)
final_columns = [
    'token_id', 'time_of_sale', 'sale_time', 'price', 'buyer', 'seller', 'event_count',
]
# Add the renamed buyer and seller columns
final_columns.extend(buyer_rename_n.values())
final_columns.extend(seller_rename_n.values())

# Select and reorder
# Use list comprehension to handle potential missing columns gracefully
df_final = df_final[[col for col in final_columns if col in df_final.columns]]

# --- Inspect and Save ---
print(df_final.head())
print(f"Final DataFrame shape: {df_final.shape}")
print("Columns:", df_final.columns)

# Save the consolidated dataframe
df_final.to_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_consolidated_buyer.csv", index=False)

# The old code that produced dfxxx.csv can be removed or commented out
# df = pd.merge(df_table1_long, df_buyer_n, left_on=['buyer','time_of_sale'],right_on=['buyer_n_address','time_of_sale'], how='left')
# df = pd.merge(df, df_seller_n, left_on=['seller','time_of_sale'],right_on=['seller_n_address','time_of_sale'], how='left')
# df = pd.merge(df, df_buyer_n_1, left_on=['buyer','time_of_sale'],right_on=['buyer_n-1_address','time_of_sale'], how='left')
# df = pd.merge(df, df_seller_n_1, left_on=['seller','time_of_sale'],right_on=['seller_n-1_address','time_of_sale'], how='left')

   token_id time_of_sale sale_time  price  \
0      1457  time_n_sale   2023-12  29.91   
1      1810  time_n_sale   2024-12  17.80   
2      3978  time_n_sale   2023-09  38.00   
3      4504  time_n_sale   2024-02  20.98   
4      4423  time_n_sale   2021-05   0.50   

                                        buyer  \
0  0x5f1ee29361206f1a129e808736f11598356c6031   
1  0x0a60cf261abef4fcd00b549f63711d0f5b9d0da4   
2  0x020ca66c30bec2c4fe3861a94e4db4a498a35872   
3  0x0097b9cfe64455eed479292671a1121f502bc954   
4  0x09a4d3be4fb596e26fbcf2e9379460349b111131   

                                       seller  event_count  buyer_tscount  \
0  0x064980edd7d43abee781f49a4e31c06df05ecccb           23        10000.0   
1  0xb65b4fd4dcb639b400fbd3d0ee5d272b5e49c2a8           20           91.0   
2  0xdb682791f6ea977db2b35ebe710863e7db1aafe5            3        10000.0   
3  0xd0f716638fd372a5ee3656b512eda6691907c3c4            2         7102.0   
4  0x33d27f0bb797de3d20e519f4a192b7570c56681b    

### Consolidated Sellers
fixing the empty rows

In [8]:
import pandas as pd
import numpy as np

# Load the long format sales data
df_table1_long = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table1_long.csv")
df_table1_long['price'] = df_table1_long['price'].round(2)

# Load the network stats data
df_buyer_n_stats = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table4.csv")
df_seller_n_stats = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table5.csv")
df_buyer_n_1_stats = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table6.csv")
df_seller_n_1_stats = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table7.csv")

# --- Rename columns in stats dataframes to generic names BEFORE merging ---
# Define the target generic column names (excluding the address column)
generic_cols_map = {
    'transaction_count': 'tscount',
    'active_period': 'act_period',
    'total_value': 'total_value',
    'total_gasUsed': 'total_gasUsed',
    'avg_gasPrice': 'avg_gasPrice',
    'avg_gasLimit': 'avg_gasLimit',
    'rolling_avg_value_last10': 'rolling_avg_value_last10',
    'rolling_std_value_last10': 'rolling_std_value_last10'
}
# Get the list of generic column values (e.g., ['tscount', 'act_period', ...])
generic_col_values = list(generic_cols_map.values())

# Function to rename stats dataframe columns
def prepare_stats_df(df, address_col_original):
    df = df.rename(columns=generic_cols_map)
    df = df.rename(columns={address_col_original: 'address'})
    # Keep only address and generic columns
    cols_to_keep = ['address'] + generic_col_values
    return df[[col for col in cols_to_keep if col in df.columns]]

# Prepare each stats dataframe
df_buyer_n_stats = prepare_stats_df(df_buyer_n_stats, 'buyer_n_address')
df_seller_n_stats = prepare_stats_df(df_seller_n_stats, 'seller_n_address')
df_buyer_n_1_stats = prepare_stats_df(df_buyer_n_1_stats, 'buyer_n-1_address')
df_seller_n_1_stats = prepare_stats_df(df_seller_n_1_stats, 'seller_n-1_address')


# --- Merge based on time_of_sale ---

# Merge for time_n_sale
df_n = df_table1_long[df_table1_long['time_of_sale'] == 'time_n_sale'].copy()
# Merge buyer stats, adding '_buyer' suffix to overlapping columns (the generic ones)
df_n = pd.merge(df_n, df_buyer_n_stats, left_on='buyer', right_on='address', how='left', suffixes=('', '_buyer'))
# Merge seller stats, adding '_seller' suffix to overlapping columns
df_n = pd.merge(df_n, df_seller_n_stats, left_on='seller', right_on='address', how='left', suffixes=('', '_seller'))

# Rename suffixed columns to final buyer_ and seller_ names
rename_map_n_buyer = {f'{col}_buyer': f'buyer_{col}' for col in generic_col_values if f'{col}_buyer' in df_n.columns}
rename_map_n_seller = {f'{col}_seller': f'seller_{col}' for col in generic_col_values if f'{col}_seller' in df_n.columns}
df_n = df_n.rename(columns=rename_map_n_buyer)
df_n = df_n.rename(columns=rename_map_n_seller)
# Drop temporary address columns from merges
df_n = df_n.drop(columns=['address_buyer', 'address_seller'], errors='ignore')


# Merge for time_n-1_sale
df_n_1 = df_table1_long[df_table1_long['time_of_sale'] == 'time_n-1_sale'].copy()
# Merge buyer stats, adding '_buyer' suffix
df_n_1 = pd.merge(df_n_1, df_buyer_n_1_stats, left_on='buyer', right_on='address', how='left', suffixes=('', '_buyer'))
# Merge seller stats, adding '_seller' suffix
df_n_1 = pd.merge(df_n_1, df_seller_n_1_stats, left_on='seller', right_on='address', how='left', suffixes=('', '_seller'))

# Rename suffixed columns to final buyer_ and seller_ names
rename_map_n1_buyer = {f'{col}_buyer': f'buyer_{col}' for col in generic_col_values if f'{col}_buyer' in df_n_1.columns}
rename_map_n1_seller = {f'{col}_seller': f'seller_{col}' for col in generic_col_values if f'{col}_seller' in df_n_1.columns}
df_n_1 = df_n_1.rename(columns=rename_map_n1_buyer)
df_n_1 = df_n_1.rename(columns=rename_map_n1_seller)
# Drop temporary address columns from merges
df_n_1 = df_n_1.drop(columns=['address_buyer', 'address_seller'], errors='ignore')


# --- Concatenate the results ---
df_final = pd.concat([df_n, df_n_1], ignore_index=True, sort=False)

# --- Clean up columns ---
# Define final columns to keep
base_columns = ['token_id', 'time_of_sale', 'sale_time', 'price', 'buyer', 'seller', 'event_count']
buyer_stat_columns = [f'buyer_{col}' for col in generic_col_values]
seller_stat_columns = [f'seller_{col}' for col in generic_col_values]

final_columns_ordered = base_columns + buyer_stat_columns + seller_stat_columns

# Select and reorder, keeping only columns that actually exist in df_final
df_final = df_final[[col for col in final_columns_ordered if col in df_final.columns]]

# --- Inspect and Save ---
print("Sample of the final DataFrame:")
print(df_final.head())
print(f"\nFinal DataFrame shape: {df_final.shape}")
print("\nColumns:", df_final.columns.tolist()) # Use tolist() for cleaner printing

# Save the consolidated dataframe
output_path = r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_consolidated_seller.csv"
df_final.to_csv(output_path, index=False)
print(f"\nSaved consolidated data to: {output_path}")

# The old code that produced dfxxx.csv can be removed or commented out
# df = pd.merge(df_table1_long, df_buyer_n, left_on=['buyer','time_of_sale'],right_on=['buyer_n_address','time_of_sale'], how='left')
# df = pd.merge(df, df_seller_n, left_on=['seller','time_of_sale'],right_on=['seller_n_address','time_of_sale'], how='left')
# df = pd.merge(df, df_buyer_n_1, left_on=['buyer','time_of_sale'],right_on=['buyer_n-1_address','time_of_sale'], how='left')
# df = pd.merge(df, df_seller_n_1, left_on=['seller','time_of_sale'],right_on=['seller_n-1_address','time_of_sale'], how='left')
# df.to_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\dfxxx.csv", index=False)

Sample of the final DataFrame:
   token_id time_of_sale sale_time  price  \
0      1457  time_n_sale   2023-12  29.91   
1      1810  time_n_sale   2024-12  17.80   
2      3978  time_n_sale   2023-09  38.00   
3      4504  time_n_sale   2024-02  20.98   
4      4423  time_n_sale   2021-05   0.50   

                                        buyer  \
0  0x5f1ee29361206f1a129e808736f11598356c6031   
1  0x0a60cf261abef4fcd00b549f63711d0f5b9d0da4   
2  0x020ca66c30bec2c4fe3861a94e4db4a498a35872   
3  0x0097b9cfe64455eed479292671a1121f502bc954   
4  0x09a4d3be4fb596e26fbcf2e9379460349b111131   

                                       seller  event_count  seller_tscount  \
0  0x064980edd7d43abee781f49a4e31c06df05ecccb           23          6468.0   
1  0xb65b4fd4dcb639b400fbd3d0ee5d272b5e49c2a8           20           681.0   
2  0xdb682791f6ea977db2b35ebe710863e7db1aafe5            3            67.0   
3  0xd0f716638fd372a5ee3656b512eda6691907c3c4            2           980.0   
4  0x33d27f0b

### merge

In [9]:
df1 = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_consolidated_buyer.csv")
df2 = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_consolidated_seller.csv")
# merge 

df = pd.merge(df1, df2, on=['token_id', 'time_of_sale', 'sale_time', 'price', 'buyer', 'seller', 'event_count'], how='left')

df.to_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_consolidated_buyer_seller.csv", index=False)

### Merge for Offer table 

In [10]:
import pandas as pd
import numpy as np
df_x = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_consolidated_buyer_seller.csv")

df_offer =  pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\offers_monthly_summary.csv")

df_x = pd.merge(df_x, df_offer,left_on=['token_id','sale_time'], right_on=['token_id', 'year_month'], how='left')
df_x.fillna(0, inplace=True)
df_x.drop(columns=['year_month'], inplace=True)
df_x

,token_id,time_of_sale,sale_time,price,buyer,seller,event_count,buyer_tscount,buyer_act_period,buyer_total_value,...,seller_rolling_avg_value_last10,seller_rolling_std_value_last10,total_offers,unique_makers_count,mean_offer_price,std_offer_price,median_offer_price,highest_offer,lowest_offer,duration_offer_days
0,1457,time_n_sale,2023-12,29.91,0x5f1ee29361206f1a129e808736f11598356c6031,0x064980edd7d43abee781f49a4e31c06df05ecccb,23,10000.0,710.667789,12438.461812,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.00,0.000000e+00,0.000000
1,1810,time_n_sale,2024-12,17.80,0x0a60cf261abef4fcd00b549f63711d0f5b9d0da4,0xb65b4fd4dcb639b400fbd3d0ee5d272b5e49c2a8,20,91.0,1278.656852,677.692527,...,0.300000,0.900000,11.0,2.0,0.007273,0.004454,0.0100,0.01,1.905000e-11,9.335046
2,3978,time_n_sale,2023-09,38.00,0x020ca66c30bec2c4fe3861a94e4db4a498a35872,0xdb682791f6ea977db2b35ebe710863e7db1aafe5,3,10000.0,1694.837454,115918.281644,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.00,0.000000e+00,0.000000
3,4504,time_n_sale,2024-02,20.98,0x0097b9cfe64455eed479292671a1121f502bc954,0xd0f716638fd372a5ee3656b512eda6691907c3c4,2,7102.0,983.839676,24997.279316,...,0.186499,0.475144,1.0,1.0,0.010000,0.000000,0.0100,0.01,1.000000e-02,0.000000
4,4423,time_n_sale,2021-05,0.50,0x09a4d3be4fb596e26fbcf2e9379460349b111131,0x33d27f0bb797de3d20e519f4a192b7570c56681b,2,1741.0,1464.084572,1296.716823,...,0.054127,0.076152,0.0,0.0,0.000000,0.000000,0.0000,0.00,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18503,4562,time_n-1_sale,2024-11,10.52,0xf15c93562bc3944a68e938ef75d2a3360d98ca57,0x2745bd23d9b23edf1ff4b695729c4804c111676c,8,2089.0,635.365845,8830.446433,...,0.025500,0.076500,2.0,1.0,0.007500,0.002500,0.0075,0.01,5.000000e-03,2.718299
18504,662,time_n-1_sale,2021-08,7.99,0xd1bea81dd97d4fcebc5b25686bdca04deff3991f,0x7c297cb2e39aaab3612e1627457acb2a3995eabb,4,1543.0,2655.924005,1250.383616,...,0.002087,0.003323,0.0,0.0,0.000000,0.000000,0.0000,0.00,0.000000e+00,0.000000
18505,6616,time_n-1_sale,2023-02,86.69,0xee3b3cc4e1a6782824202e48c2d8b93922882a3a,0x8ad272ac86c6c88683d9a60eb8ed57e6c304bb0c,2,1003.0,1169.674931,3910.395673,...,24.330000,49.046999,0.0,0.0,0.000000,0.000000,0.0000,0.00,0.000000e+00,0.000000
18506,5566,time_n-1_sale,2021-05,2.50,0x27a7f6fca0431c02c3c5fb6116ff019a25be526d,0x37c701ecd86cdddba614cf194d9a41d1f9c55145,2,1102.0,1452.326389,1364.835291,...,0.423000,1.112367,0.0,0.0,0.000000,0.000000,0.0000,0.00,0.000000e+00,0.000000


In [11]:
# merge for traits 
df_traits = pd.read_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\df_table3.csv")

df_x = pd.merge(df_x, df_traits, left_on=['token_id'], right_on=['token_id'], how='left')
df_x
df_x.fillna(0, inplace=True)

df_x.to_csv(r"C:\Emory\Research\NFT\DataAnalysis\Model\Analysis_NFT_Sales\Panel_for_Model2.csv", index=False)